In [1]:
%matplotlib notebook
import numpy as np
import time
import scipy as sp

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from keras.models import Sequential
from keras.layers import Dense, RNN, GRU, Conv2D, Flatten, MaxPooling2D, Dropout, Input, Reshape

class MatrixMaker:
    __slots__ = ("rows", "cols", "kernel_size", "line_size", 'num_per_mat', 'smooth_matrix', 'line_start_position', 'alfa', 'matrix_fade', 'line_pos_mat')
    def __init__(self, rows, cols=None, kernel_size=(1, 1), line_size=(1, 2), num_per_mat=10):
        self.rows = rows
        self.cols = cols or rows
        self.kernel_size = kernel_size
        self.line_size = line_size
        self.num_per_mat = num_per_mat

        self.smooth_matrix = self.create_smoothed_matrix()
        self.line_start_position = self.create_line()
        self.alfa = self.create_alfa()
        self.matrix_fade = self.create_matrix_line_fade()
        self.line_pos_mat = self.create_line_pos_mat()

    def create_smoothed_matrix(self):
        kernel = np.ones(shape=self.kernel_size, dtype=float) / (self.kernel_size[0] * self.kernel_size[1])
        return sp.ndimage.convolve(np.random.rand(self.rows, self.cols), kernel)

    def create_line(self):
        return (np.random.randint(low=0, high=self.rows - self.line_size[0] + 1),
                np.random.randint(low=0, high=self.cols - self.line_size[1] + 1))

    def create_alfa(self):
        return np.linspace(1, 0, num=self.num_per_mat)

    def create_matrix_with_line(self, alfa):
        matrix = np.ones((self.rows, self.cols))
        matrix[self.line_start_position[0]:self.line_start_position[0] + self.line_size[0],
        self.line_start_position[1]:self.line_start_position[1] + self.line_size[1]] = alfa
        return matrix

    def create_matrix_line_fade(self):
        matrix_line_fade = []
        for i in range(self.num_per_mat):
            line = self.create_matrix_with_line(self.alfa[i])
            matrix_line_fade.append(self.smooth_matrix * line)

        return matrix_line_fade

    def create_line_pos_mat(self):
        return np.array(np.logical_not(self.create_matrix_with_line(0)).astype(int), dtype='float32')


class MatrixLister:
    def __init__(self, row_len, col_len, kernel_size, min_max_line_size, num_of_mat, num_per_mat, num_neuron):
        self.row_len = row_len
        self.col_len = col_len
        self.kernel_size = kernel_size
        self.min_max_line_size = min_max_line_size
        self.num_of_mat = num_of_mat
        self.num_per_mat = num_per_mat

        self.matrix_list = self.create_matrix_in_list()
        self.con_matrix, self.con_alfa = self.concatenate_matrices()

        self.neural_network = NeuralNetwork(input_size=(row_len, col_len), num_neuron=num_neuron)

    def create_matrix_in_list(self):
        line_sizes = [(np.random.randint(self.min_max_line_size[0][0], self.min_max_line_size[1][0] + 1),
                       np.random.randint(self.min_max_line_size[0][1], self.min_max_line_size[1][1] + 1))
                      for _ in range(self.num_of_mat)]

        return [MatrixMaker(self.row_len, self.col_len, self.kernel_size, line_sizes[i], self.num_per_mat)
                for i in range(self.num_of_mat)]

    def concatenate_matrices(self):
        concatenated_matrices = []
        con_alfa = []

        for matrix in self.matrix_list:
            concatenated_matrices += matrix.create_matrix_line_fade()
            con_alfa += list(matrix.alfa)

        return np.array(concatenated_matrices, dtype='float32'), con_alfa

    def con_line_pos_mat(self):
        con_line_pos_mat = []
        for matrix in self.matrix_list:
            for alfa in matrix.alfa:
                if alfa != 1:
                    con_line_pos_mat.append(matrix.line_pos_mat)
                else:
                    con_line_pos_mat.append(np.zeros((self.row_len, self.col_len)))
                    
                    
        fisk = np.stack(con_line_pos_mat)
        return fisk

    def train_neural_network(self, num_epochs=10, batch_size=64):

        # Assuming you have input_data and output_data for training
        #output_data = np.array(self.con_line_pos_mat()).reshape(len(self.con_line_pos_mat()), -1)

        self.neural_network.train(self.con_matrix, self.con_line_pos_mat(), num_epochs, batch_size)

    def plot(self, interval=200):
        fig, ax = plt.subplots()

        def update(frame):
            ax.clear()
            im = ax.imshow(self.con_matrix[frame], interpolation='nearest', aspect='auto', vmin=0, vmax=1)

            return [im]

        animation = FuncAnimation(fig, update, frames=len(self.con_matrix), interval=interval, repeat=False, blit=True)
        plt.tight_layout()
        plt.show(block=False)
        plt.show()
        return animation
    
    def plot_matrices(self, num_to_pred, new_mat=False, interval=200):
        if new_mat:
            self.num_of_mat = num_to_pred
            self.matrix_list = self.create_matrix_in_list()
            self.con_matrix, self.con_alfa = self.concatenate_matrices()
            
        input_matrix = np.array(self.con_matrix)
        true_matrix = np.array(self.con_line_pos_mat())
        pred = self.neural_network.predict(input_matrix)
    
        predicted_line_pos_mat = np.array(pred).reshape(input_matrix.shape)
        
        fig, axes = plt.subplots(1, 3, figsize=(12, 4))  # 1 row, 3 columns
    
        def update(frame):
            # Plot Input Matrix
            im = [axes[0].imshow(input_matrix[frame], interpolation='nearest', aspect='auto', vmin=0, vmax=1)]
            axes[0].set_title('Input Matrix')
    
            # Plot True Line Position Matrix
            im.append(axes[1].imshow(true_matrix[frame], interpolation='nearest', aspect='auto', vmin=0, vmax=1))
            axes[1].set_title('True Line Position Matrix')
    
            # Plot Predicted Line Position Matrix
            im.append(axes[2].imshow(predicted_line_pos_mat[frame], interpolation='nearest', aspect='auto', vmin=0, vmax=1))
            axes[2].set_title('Predicted Line Position Matrix')
            
            return im
    
        animation = FuncAnimation(fig, update, frames=len(self.con_matrix), interval=interval, repeat=False, blit=True)
    
        plt.tight_layout()
        plt.show(block=False)
        plt.show()
        plt.show()
        return animation

    def testprint(self):
        return

def plot_other(other, interval=200):
    fig, ax = plt.subplots()

    def update(frame):
        ax.clear()
        im = ax.imshow(other[frame], interpolation='nearest', aspect='auto', vmin=0, vmax=1)

        return [im]

    animation = FuncAnimation(fig, update, frames=len(other), interval=interval, repeat=False, blit=True)
    plt.tight_layout()
    plt.show(block=False)
    plt.show()
    return animation


class NeuralNetwork:
    def __init__(self, input_size, num_neuron):
        self.fh = None 
        self.model = self.build_model(input_size, num_neuron)

    def build_model(self, input_size, num_neuron):
        self.fh = 1
        
        model = Sequential([
            Input(shape=(input_size[0],input_size[1],1)),
            Conv2D(64, kernel_size=(3,3), padding='same', activation='relu'),
            Flatten(),
            Dense(num_neuron, activation='relu'),
            Dense(input_size[0]*input_size[1], activation='sigmoid'),
            Reshape((input_size[0],input_size[1],1))
        ])
        
        
        model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
        return model

    def train(self, input_data, output_data, epochs, batch_size):
        in_data = np.expand_dims(input_data, -1)
        out_data = np.expand_dims(output_data, -1)

        self.model.fit(in_data, out_data, epochs=epochs, batch_size=batch_size)

    def predict(self, input_matrices):
        # Flatten and concatenate all input matrices in the batch
        input_data = np.array([matrix for matrix in input_matrices])
        print(input_data.shape)
    
        # Predict the output for the entire batch
        predicted_output = self.model.predict(input_data)
        print(predicted_output.shape)
        
        
    
        return predicted_output



def plot_matrices(input_matrix, true_matrix, predicted_line_pos_mat):
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))  # 1 row, 3 columns

    # Plot Input Matrix
    axes[0].imshow(input_matrix, cmap='viridis', interpolation='nearest', aspect='auto')
    axes[0].set_title('Input Matrix')

    # Plot True Line Position Matrix
    axes[1].imshow(true_matrix, cmap='viridis', interpolation='nearest', aspect='auto')
    axes[1].set_title('True Line Position Matrix')

    # Plot Predicted Line Position Matrix
    axes[2].imshow(predicted_line_pos_mat, cmap='viridis', interpolation='nearest', aspect='auto')
    axes[2].set_title('Predicted Line Position Matrix')

    plt.tight_layout()
    plt.show()
    

In [2]:
row_len = 10
col_len = 12
kernel_size = (3, 3)
min_max_line_size = [(2,4),(5,8)]
num_of_mat = 200
numb_of_picture = 5
num_of_neurons = 256

matrix_lister = MatrixLister(row_len, col_len, kernel_size, min_max_line_size, num_of_mat, numb_of_picture, num_of_neurons)

In [3]:
batch_size = 128
epochs = 50

start = time.time()
matrix_lister.train_neural_network(batch_size=batch_size, num_epochs=epochs)
print(time.time() - start)

Epoch 1/50
8/8 [==============================] - 1s 10ms/step - loss: 0.1846 - accuracy: 0.7447
Epoch 2/50
8/8 [==============================] - 0s 9ms/step - loss: 0.1188 - accuracy: 0.8557
Epoch 3/50
8/8 [==============================] - 0s 9ms/step - loss: 0.1136 - accuracy: 0.8599
Epoch 4/50
8/8 [==============================] - 0s 9ms/step - loss: 0.1084 - accuracy: 0.8605
Epoch 5/50
8/8 [==============================] - 0s 9ms/step - loss: 0.1021 - accuracy: 0.8672
Epoch 6/50
8/8 [==============================] - 0s 9ms/step - loss: 0.0952 - accuracy: 0.8804
Epoch 7/50
8/8 [==============================] - 0s 9ms/step - loss: 0.0861 - accuracy: 0.8917
Epoch 8/50
8/8 [==============================] - 0s 9ms/step - loss: 0.0771 - accuracy: 0.9041
Epoch 9/50
8/8 [==============================] - 0s 9ms/step - loss: 0.0689 - accuracy: 0.9132
Epoch 10/50
8/8 [==============================] - 0s 12ms/step - loss: 0.0623 - accuracy: 0.9206
Epoch 11/50
8/8 [====================

In [4]:
matrix_lister.plot_matrices(1) 

(1000, 10, 12)
32/32 [==============================] - 0s 1ms/step
(1000, 10, 12, 1)


<IPython.core.display.Javascript object>